# 05. 데이터 전처리, 추가, 삭제, 변환(with타이타닉호, 공공 데이터)

이번 장에서는 Pandas DataFrame의 row, column의 추가, 삭제, 컬럼간 연산, 타입의 변환 그리고 데이터 전처리 방법에 대하여 다뤄 보도록 합니다.

전처리 방법 파트에서는 datetime 데이터 타입을 활용하여 시간 데이터의 전처리 방법 그리고 Pandas에서 제공하는 유용한 기능에 대하여 알아봅니다.

타입 변환에서는 to_numeric을 사용하여 수치형(numerical) 데이터로 변환하는 방법과 자주 발생하는 오류에 대하여 알아봅니다.

그리고 마지막으로 cut과 qcut의 차이점에 대하여 살펴보고 이를 활용하여 연속된 수치형 데이터를 binning 하여 카테고리화 하는 방법에 대해서도 다뤄 봅니다.

### 모듈 import

In [1]:
from IPython.display import Image
import numpy as np
import pandas as pd
import seaborn as sns
import warnings

# warning 무시
warnings.filterwarnings('ignore')

# e notation 표현 방식 변경
pd.options.display.float_format = '{:.2f}'.format

# 모든 컬럼 표시
pd.set_option('display.max_columns', None)

### 데이터셋 로드

In [2]:
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.00,1,0,7.25,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.00,1,0,71.28,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.00,0,0,7.92,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.00,1,0,53.10,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.00,0,0,8.05,S,Third,man,True,NaN,Southampton,no,True


컬럼(columns) 설명

- survivied: 생존여부 (1: 생존, 0: 사망)

- pclass: 좌석 등급 (1등급, 2등급, 3등급)

- sex: 성별

- age: 나이

- sibsp: 형제 + 배우자 수

- parch: 부모 + 자녀 수

- fare: 좌석 요금

- embarked: 탑승 항구 (S, C, Q)

- class: pclass와 동일

- who: 남자(man), 여자(woman), 아이(child)

- adult_male: 성인 남자 여부

- deck: 데크 번호 (알파벳 + 숫자 혼용)

- embark_town: 탑승 항구 이름

- alive: 생존여부 (yes, no)

- alone: 혼자 탑승 여부

---

### 새로운 컬럼 추가

In [3]:
df1 = df.copy()
df1.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.00,1,0,7.25,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.00,1,0,71.28,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.00,0,0,7.92,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.00,1,0,53.10,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.00,0,0,8.05,S,Third,man,True,NaN,Southampton,no,True


임의의 값을 대입하여 새로운 컬럼을 추가할 수 있습니다.

In [4]:
df1['VIP'] = True
df1.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,VIP
0,0,3,male,22.00,1,0,7.25,S,Third,man,True,NaN,Southampton,no,False,True
1,1,1,female,38.00,1,0,71.28,C,First,woman,False,C,Cherbourg,yes,False,True
2,1,3,female,26.00,0,0,7.92,S,Third,woman,False,NaN,Southampton,yes,True,True
3,1,1,female,35.00,1,0,53.10,S,First,woman,False,C,Southampton,yes,False,True
4,0,3,male,35.00,0,0,8.05,S,Third,man,True,NaN,Southampton,no,True,True


중간에 컬럼을 추가하고 싶은 경우 insert()를 활용할 수 있습니다.  
insert(컬럼인덱스, 컬럼명, 값)

In [5]:
df1.insert(5, 'RICH', df1['fare'] > 100)
df1.head()

,survived,pclass,sex,age,sibsp,RICH,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,VIP
0,0,3,male,22.00,1,False,0,7.25,S,Third,man,True,NaN,Southampton,no,False,True
1,1,1,female,38.00,1,False,0,71.28,C,First,woman,False,C,Cherbourg,yes,False,True
2,1,3,female,26.00,0,False,0,7.92,S,Third,woman,False,NaN,Southampton,yes,True,True
3,1,1,female,35.00,1,False,0,53.10,S,First,woman,False,C,Southampton,yes,False,True
4,0,3,male,35.00,0,False,0,8.05,S,Third,man,True,NaN,Southampton,no,True,True


### 삭제

삭제는 행(row) 삭제와 열(column) 삭제로 구분할 수 있습니다.

### 행 (row) 삭제

행 삭제시 index를 지정하여 삭제합니다.

In [6]:
df1.drop(1)

,survived,pclass,sex,age,sibsp,RICH,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,VIP
0,0,3,male,22.00,1,False,0,7.25,S,Third,man,True,NaN,Southampton,no,False,True
2,1,3,female,26.00,0,False,0,7.92,S,Third,woman,False,NaN,Southampton,yes,True,True
3,1,1,female,35.00,1,False,0,53.10,S,First,woman,False,C,Southampton,yes,False,True
4,0,3,male,35.00,0,False,0,8.05,S,Third,man,True,NaN,Southampton,no,True,True
5,0,3,male,NaN,0,False,0,8.46,Q,Third,man,True,NaN,Queenstown,no,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.00,0,False,0,13.00,S,Second,man,True,NaN,Southampton,no,True,True
887,1,1,female,19.00,0,False,0,30.00,S,First,woman,False,B,Southampton,yes,True,True
888,0,3,female,NaN,1,False,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,True
889,1,1,male,26.00,0,False,0,30.00,C,First,man,True,C,Cherbourg,yes,True,True


행 삭제시 범위를 지정하여 삭제할 수 있습니다.

In [7]:
df1.drop(np.arange(10))

,survived,pclass,sex,age,sibsp,RICH,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,VIP
10,1,3,female,4.00,1,False,1,16.70,S,Third,child,False,G,Southampton,yes,False,True
11,1,1,female,58.00,0,False,0,26.55,S,First,woman,False,C,Southampton,yes,True,True
12,0,3,male,20.00,0,False,0,8.05,S,Third,man,True,NaN,Southampton,no,True,True
13,0,3,male,39.00,1,False,5,31.27,S,Third,man,True,NaN,Southampton,no,False,True
14,0,3,female,14.00,0,False,0,7.85,S,Third,child,False,NaN,Southampton,no,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.00,0,False,0,13.00,S,Second,man,True,NaN,Southampton,no,True,True
887,1,1,female,19.00,0,False,0,30.00,S,First,woman,False,B,Southampton,yes,True,True
888,0,3,female,NaN,1,False,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,True
889,1,1,male,26.00,0,False,0,30.00,C,First,man,True,C,Cherbourg,yes,True,True


fancy indexing을 활용하여 삭제할 수 있습니다.

In [8]:
df1.drop([1, 3, 5, 7, 9])

,survived,pclass,sex,age,sibsp,RICH,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,VIP
0,0,3,male,22.00,1,False,0,7.25,S,Third,man,True,NaN,Southampton,no,False,True
2,1,3,female,26.00,0,False,0,7.92,S,Third,woman,False,NaN,Southampton,yes,True,True
4,0,3,male,35.00,0,False,0,8.05,S,Third,man,True,NaN,Southampton,no,True,True
6,0,1,male,54.00,0,False,0,51.86,S,First,man,True,E,Southampton,no,True,True
8,1,3,female,27.00,0,False,2,11.13,S,Third,woman,False,NaN,Southampton,yes,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.00,0,False,0,13.00,S,Second,man,True,NaN,Southampton,no,True,True
887,1,1,female,19.00,0,False,0,30.00,S,First,woman,False,B,Southampton,yes,True,True
888,0,3,female,NaN,1,False,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,True
889,1,1,male,26.00,0,False,0,30.00,C,First,man,True,C,Cherbourg,yes,True,True


### 열 (column) 삭제

In [9]:
df1.head()

,survived,pclass,sex,age,sibsp,RICH,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,VIP
0,0,3,male,22.00,1,False,0,7.25,S,Third,man,True,NaN,Southampton,no,False,True
1,1,1,female,38.00,1,False,0,71.28,C,First,woman,False,C,Cherbourg,yes,False,True
2,1,3,female,26.00,0,False,0,7.92,S,Third,woman,False,NaN,Southampton,yes,True,True
3,1,1,female,35.00,1,False,0,53.10,S,First,woman,False,C,Southampton,yes,False,True
4,0,3,male,35.00,0,False,0,8.05,S,Third,man,True,NaN,Southampton,no,True,True


열 삭제시 반드시 axis=1 옵션을 지정해야 합니다. 2번째 위치에 지정시 axis=을 생략할 수 있습니다.

In [10]:
df1.drop('class', axis=1).head()

,survived,pclass,sex,age,sibsp,RICH,parch,fare,embarked,who,adult_male,deck,embark_town,alive,alone,VIP
0,0,3,male,22.00,1,False,0,7.25,S,man,True,NaN,Southampton,no,False,True
1,1,1,female,38.00,1,False,0,71.28,C,woman,False,C,Cherbourg,yes,False,True
2,1,3,female,26.00,0,False,0,7.92,S,woman,False,NaN,Southampton,yes,True,True
3,1,1,female,35.00,1,False,0,53.10,S,woman,False,C,Southampton,yes,False,True
4,0,3,male,35.00,0,False,0,8.05,S,man,True,NaN,Southampton,no,True,True


In [ ]:
df1.drop('class', 1).head()

TypeError: drop() takes from 1 to 2 positional arguments but 3 were given

다수의 컬럼(column) 삭제도 가능합니다.

In [14]:
df1.drop(['who', 'deck', 'alive'], axis=1)

,survived,pclass,sex,age,sibsp,RICH,parch,fare,embarked,class,adult_male,embark_town,alone,VIP
0,0,3,male,22.00,1,False,0,7.25,S,Third,True,Southampton,False,True
1,1,1,female,38.00,1,False,0,71.28,C,First,False,Cherbourg,False,True
2,1,3,female,26.00,0,False,0,7.92,S,Third,False,Southampton,True,True
3,1,1,female,35.00,1,False,0,53.10,S,First,False,Southampton,False,True
4,0,3,male,35.00,0,False,0,8.05,S,Third,True,Southampton,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.00,0,False,0,13.00,S,Second,True,Southampton,True,True
887,1,1,female,19.00,0,False,0,30.00,S,First,False,Southampton,True,True
888,0,3,female,NaN,1,False,2,23.45,S,Third,False,Southampton,False,True
889,1,1,male,26.00,0,False,0,30.00,C,First,True,Cherbourg,True,True


삭제된 내용을 바로 적용하려면

1. inplace=True를 지정합니다.

2. 변수에 재대입 하여 결과를 반영합니다.

In [15]:
df1.drop(['who', 'deck', 'alive'], axis=1, inplace=True)
df1.head()

,survived,pclass,sex,age,sibsp,RICH,parch,fare,embarked,class,adult_male,embark_town,alone,VIP
0,0,3,male,22.00,1,False,0,7.25,S,Third,True,Southampton,False,True
1,1,1,female,38.00,1,False,0,71.28,C,First,False,Cherbourg,False,True
2,1,3,female,26.00,0,False,0,7.92,S,Third,False,Southampton,True,True
3,1,1,female,35.00,1,False,0,53.10,S,First,False,Southampton,False,True
4,0,3,male,35.00,0,False,0,8.05,S,Third,True,Southampton,True,True


--- 

### 컬럼간 연산

컬럼(column) 과 컬럼 사이의 연산을 매우 쉽게 적용할 수 있습니다.

In [16]:
# df 데이터프레임 복제
df1 = df.copy()

family(가족)의 총합은 sibsp컬럼과 parch의 합산으로 구할 수 있습니다.

In [17]:
df1['family'] = df1['sibsp'] + df1['parch']
df1.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,family
0,0,3,male,22.00,1,0,7.25,S,Third,man,True,NaN,Southampton,no,False,1
1,1,1,female,38.00,1,0,71.28,C,First,woman,False,C,Cherbourg,yes,False,1
2,1,3,female,26.00,0,0,7.92,S,Third,woman,False,NaN,Southampton,yes,True,0
3,1,1,female,35.00,1,0,53.10,S,First,woman,False,C,Southampton,yes,False,1
4,0,3,male,35.00,0,0,8.05,S,Third,man,True,NaN,Southampton,no,True,0


문자열의 합 (이어붙히기)도 가능합니다.

In [18]:
df1['gender'] = df1['who'] + '-' + df1['sex']
df1.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,family,gender
0,0,3,male,22.00,1,0,7.25,S,Third,man,True,NaN,Southampton,no,False,1,man-male
1,1,1,female,38.00,1,0,71.28,C,First,woman,False,C,Cherbourg,yes,False,1,woman-female
2,1,3,female,26.00,0,0,7.92,S,Third,woman,False,NaN,Southampton,yes,True,0,woman-female
3,1,1,female,35.00,1,0,53.10,S,First,woman,False,C,Southampton,yes,False,1,woman-female
4,0,3,male,35.00,0,0,8.05,S,Third,man,True,NaN,Southampton,no,True,0,man-male


컬럼간 연산시 round()를 사용하여 소수점 자릿수를 지정할 수 있습니다.

round(숫자, 소수 몇 째자리)

In [19]:
df1['round'] = round(df1['fare'] / df1['age'], 2)
df1.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,family,gender,round
0,0,3,male,22.00,1,0,7.25,S,Third,man,True,NaN,Southampton,no,False,1,man-male,0.33
1,1,1,female,38.00,1,0,71.28,C,First,woman,False,C,Cherbourg,yes,False,1,woman-female,1.88
2,1,3,female,26.00,0,0,7.92,S,Third,woman,False,NaN,Southampton,yes,True,0,woman-female,0.30
3,1,1,female,35.00,1,0,53.10,S,First,woman,False,C,Southampton,yes,False,1,woman-female,1.52
4,0,3,male,35.00,0,0,8.05,S,Third,man,True,NaN,Southampton,no,True,0,man-male,0.23


연산시 1개의 컬럼이라도 NaN 값을 포함하고 있다면 결과는 NaN 이 됩니다.

In [20]:
df1.loc[df1['age'].isnull(), 'deck':].head()

,deck,embark_town,alive,alone,family,gender,round
5,NaN,Queenstown,no,True,0,man-male,NaN
17,NaN,Southampton,yes,True,0,man-male,NaN
19,NaN,Cherbourg,yes,True,0,woman-female,NaN
26,NaN,Cherbourg,no,True,0,man-male,NaN
28,NaN,Queenstown,yes,True,0,woman-female,NaN


---

### category 타입

In [21]:
df1 = df.copy()
df1.head(2)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.00,1,0,7.25,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.00,1,0,71.28,C,First,woman,False,C,Cherbourg,yes,False


In [22]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


category로 변경

category로 변경시에는 Categories가 같이 출력 됩니다.

In [24]:
df1['who'].astype('category').head()

0      man
1    woman
2    woman
3    woman
4      man
Name: who, dtype: category
Categories (3, object): ['child', 'man', 'woman']

변경사항을 적용합니다.

In [25]:
df1['who'] = df1['who'].astype('category')

category 타입으로 변경시 사용하는 메모리도 감소합니다.

In [26]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    category
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(3), float64(2), int64(4), object(4)
memory usage: 74.7+ KB


타입을 category로 변환했다면 .cat으로 접근하여 category 타입이 제공하는 attribute를 사용할 수 있습니다.

카테고리 출력

In [29]:
df1['who'].cat.categories

Index(['child', 'man', 'woman'], dtype='object')

카테고리 이름 변경

- 변경시 원래 순서에 맞게 입력

In [31]:
# df1['who'].cat.categories = ['아이', '남자', '여자']
df1['who'] = df1['who'].cat.rename_categories(['아이', '남자', '여자'])
df1['who'].value_counts()

who
남자    537
여자    271
아이     83
Name: count, dtype: int64

---

### datetime - 날짜, 시간

### datetime 타입이 가지는 고유 기능

datetime 타입에서는 dt 접근자로 다음과 같은 날짜 속성에 쉽게 접근할 수 있습니다.

Pandas의 dt (datetime) 날짜 관련 변수는 다음과 같습니다.

- pandas.Series.dt.year: 연도

- pandas.Series.dt.month: 월

- pandas.Series.dt.day: 일

- pandas.Series.dt.hour: 시

- pandas.Series.dt.minute: 분

- pandas.Series.dt.second: 초

- pandas.Series.dt.microsecond: micro 초

- pandas.Series.dt.nanosecond: nano 초

- pandas.Series.dt.week: 주

- pandas.Series.dt.weekofyear: 연중 몇 째주

- pandas.Series.dt.dayofweek: 요일

- pandas.Series.dt.weekday: 요일 (dayofweek과 동일)

- pandas.Series.dt.dayofyear: 연중 몇 번째 날

- pandas.Series.dt.quarter: 분기

### to_datetime

샘플용 서울시 공공자전거 데이터를 로드합니다.

In [32]:
# 실습용 데이터셋 다운로드
!pip install opendata-kr -q

from opendata import dataset

dataset.download('서울시자전거')

[서버] Jaen
======= 다운로드 시작 =======

data/서울시자전거/seoul_bicycle.csv


  0%|          | 0.00/31.1M [00:00<?, ?B/s]


======= 다운로드 완료 =======


In [34]:
# 데이터셋 로드
df2 = pd.read_csv('data/서울시자전거/seoul_bicycle.csv')
df2.head()

,대여일자,대여소번호,대여소명,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리,이용시간
0,Jan-20-2020,3,중랑센터,일일(회원),M,AGE_003,3,61.82,0.52,2230.00,75
1,Jan-20-2020,3,중랑센터,일일(회원),M,AGE_004,1,39.62,0.28,1220.00,15
2,Jan-20-2020,3,중랑센터,정기,M,AGE_005,3,430.85,4.01,17270.00,53
3,Jan-20-2020,5,상암센터 정비실,일일(회원),\N,AGE_005,2,1.79,0.02,90.00,33
4,Jan-20-2020,5,상암센터 정비실,정기,F,AGE_003,1,4501.96,45.47,196010.00,64


In [35]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327231 entries, 0 to 327230
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   대여일자    327231 non-null  object 
 1   대여소번호   327231 non-null  int64  
 2   대여소명    327231 non-null  object 
 3   대여구분코드  327231 non-null  object 
 4   성별      272841 non-null  object 
 5   연령대코드   327231 non-null  object 
 6   이용건수    327231 non-null  int64  
 7   운동량     327231 non-null  object 
 8   탄소량     327231 non-null  object 
 9   이동거리    327231 non-null  float64
 10  이용시간    327231 non-null  int64  
dtypes: float64(1), int64(3), object(7)
memory usage: 27.5+ MB


대여일자 컬럼은 날짜 관련 컬럼처럼 보이나 info()는 object로 인식하였습니다.

datetime타입으로 변경해야 .dt 접근자를 사용할 수 있습니다.

pd.to_datetime() : datetime type으로 변환합니다.

In [36]:
pd.to_datetime(df2['대여일자'])

0        2020-01-20
1        2020-01-20
2        2020-01-20
3        2020-01-20
4        2020-01-20
            ...    
327226   2020-05-20
327227   2020-05-20
327228   2020-05-20
327229   2020-05-20
327230   2020-05-20
Name: 대여일자, Length: 327231, dtype: datetime64[ns]

재대입하여 컬럼에 적용합니다.

In [37]:
df2['대여일자'] = pd.to_datetime(df2['대여일자'])

df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327231 entries, 0 to 327230
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   대여일자    327231 non-null  datetime64[ns]
 1   대여소번호   327231 non-null  int64         
 2   대여소명    327231 non-null  object        
 3   대여구분코드  327231 non-null  object        
 4   성별      272841 non-null  object        
 5   연령대코드   327231 non-null  object        
 6   이용건수    327231 non-null  int64         
 7   운동량     327231 non-null  object        
 8   탄소량     327231 non-null  object        
 9   이동거리    327231 non-null  float64       
 10  이용시간    327231 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 27.5+ MB


적용된 후 .dt 접근자를 활용하여 datetime 속성에 접근할 수 있습니다.

In [38]:
df2['대여일자'].dt.year

0         2020
1         2020
2         2020
3         2020
4         2020
          ... 
327226    2020
327227    2020
327228    2020
327229    2020
327230    2020
Name: 대여일자, Length: 327231, dtype: int32

In [39]:
df2['대여일자'].dt.month

0         1
1         1
2         1
3         1
4         1
         ..
327226    5
327227    5
327228    5
327229    5
327230    5
Name: 대여일자, Length: 327231, dtype: int32

In [40]:
df2['대여일자'].dt.day

0         20
1         20
2         20
3         20
4         20
          ..
327226    20
327227    20
327228    20
327229    20
327230    20
Name: 대여일자, Length: 327231, dtype: int32

In [41]:
df2['대여일자'].dt.dayofweek

0         0
1         0
2         0
3         0
4         0
         ..
327226    2
327227    2
327228    2
327229    2
327230    2
Name: 대여일자, Length: 327231, dtype: int32

---

### pd.cut() - 구간 나누기(binning)

연속된 수치(continuous values)를 구간으로 나누어 카테고리화 할 때 사용합니다.

In [42]:
df2.head()

,대여일자,대여소번호,대여소명,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리,이용시간
0,2020-01-20,3,중랑센터,일일(회원),M,AGE_003,3,61.82,0.52,2230.00,75
1,2020-01-20,3,중랑센터,일일(회원),M,AGE_004,1,39.62,0.28,1220.00,15
2,2020-01-20,3,중랑센터,정기,M,AGE_005,3,430.85,4.01,17270.00,53
3,2020-01-20,5,상암센터 정비실,일일(회원),\N,AGE_005,2,1.79,0.02,90.00,33
4,2020-01-20,5,상암센터 정비실,정기,F,AGE_003,1,4501.96,45.47,196010.00,64


직접 범위 설정을 해줄 수 있습니다.

right=False로 지정시 우측 범위를 포함하지 않습니다.

In [43]:
bins = [0, 6000, 100000, df2['이동거리'].max()]
pd.cut(df2['이동거리'], bins, right=False)

0                   [0.0, 6000.0)
1                   [0.0, 6000.0)
2              [6000.0, 100000.0)
3                   [0.0, 6000.0)
4         [100000.0, 56709052.94)
                   ...           
327226         [6000.0, 100000.0)
327227              [0.0, 6000.0)
327228              [0.0, 6000.0)
327229              [0.0, 6000.0)
327230    [100000.0, 56709052.94)
Name: 이동거리, Length: 327231, dtype: category
Categories (3, interval[float64, left]): [[0.0, 6000.0) < [6000.0, 100000.0) < [100000.0, 56709052.94)]

labels를 지정해 줄 수 있으며, 지정한 bins의 개수보다 1 개가 적어야 합니다.

In [44]:
labels = ['적음', '보통', '많음']

pd.cut(df2['이동거리'], bins, labels=labels, right=False)

0         적음
1         적음
2         보통
3         적음
4         많음
          ..
327226    보통
327227    적음
327228    적음
327229    적음
327230    많음
Name: 이동거리, Length: 327231, dtype: category
Categories (3, object): ['적음' < '보통' < '많음']

pd.cut()을 활용하여 쉽게 그룹을 나눌 수 있습니다.

In [45]:
df2.head()

,대여일자,대여소번호,대여소명,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리,이용시간
0,2020-01-20,3,중랑센터,일일(회원),M,AGE_003,3,61.82,0.52,2230.00,75
1,2020-01-20,3,중랑센터,일일(회원),M,AGE_004,1,39.62,0.28,1220.00,15
2,2020-01-20,3,중랑센터,정기,M,AGE_005,3,430.85,4.01,17270.00,53
3,2020-01-20,5,상암센터 정비실,일일(회원),\N,AGE_005,2,1.79,0.02,90.00,33
4,2020-01-20,5,상암센터 정비실,정기,F,AGE_003,1,4501.96,45.47,196010.00,64


bins 옵션에 나누고자 하는 구간의 개수를 설정합니다.

In [46]:
df2['이동거리_cut'] = pd.cut(df2['이동거리'], bins=3)

df2['이동거리_cut'].value_counts()

이동거리_cut
(-56709.053, 18903017.647]      327216
(18903017.647, 37806035.293]        12
(37806035.293, 56709052.94]          3
Name: count, dtype: int64

분포를 보니 첫 구간에 대부분의 데이터가 쏠려 있습니다. 딱봐도 올바르지 않은 방법 같아 보입니다.

pd.cut()은 최소에서 최대 구간을 지정한 bin만큼 동일하게 분할 하기 때문에 이런 현상이 발생할 수 있습니다.

고르게 분포한 데이터라면 괜찮지만, 튀는 이상치(outlier)가 있는 경우에는 안 좋은 결과를 초래 합니다.

---

### pd.qcut() - 동일한 갯수를 갖도록 구간 분할

pd.cut()과 유사하지만, quantity 즉 데이터의 분포를 최대한 비슷하게 유지하는 구간을 분할 합니다.

In [47]:
df2['이동거리_qcut'] = pd.qcut(df2['이동거리'], q=3)

df2['이동거리_qcut'].value_counts()

이동거리_qcut
(-0.001, 9030.0]          109095
(60470.0, 56709052.94]    109072
(9030.0, 60470.0]         109064
Name: count, dtype: int64

구간도 예쁘게 분할(균등하게 분할)이 된 것 처럼 보입니다. 하지만, 간격은 일정하지 않습니다.

qcut 또한 임의 범위를 조정할 수 있습니다.

In [48]:
qcut_bins = [0, 0.2, 0.8, 1]

pd.qcut(df2['이동거리'], qcut_bins)

0                (-0.001, 3570.0]
1                (-0.001, 3570.0]
2              (3570.0, 125530.0]
3                (-0.001, 3570.0]
4         (125530.0, 56709052.94]
                   ...           
327226         (3570.0, 125530.0]
327227           (-0.001, 3570.0]
327228           (-0.001, 3570.0]
327229           (-0.001, 3570.0]
327230    (125530.0, 56709052.94]
Name: 이동거리, Length: 327231, dtype: category
Categories (3, interval[float64, right]): [(-0.001, 3570.0] < (3570.0, 125530.0] < (125530.0, 56709052.94]]

qcut 역시 label을 지정할 수 있습니다. 마찬가지로 범위 수보다 1개 적게 설정합니다.

In [49]:
qcut_labels = ['적음', '보통', '많음']

pd.qcut(df2['이동거리'], qcut_bins, labels=qcut_labels).value_counts()

이동거리
보통    196307
적음     65482
많음     65442
Name: count, dtype: int64

## 연습문제

In [50]:
from IPython.display import Image
import numpy as np
import pandas as pd
import seaborn as sns
import warnings

# warning 무시
warnings.filterwarnings('ignore')

# e notation 표현 방식 변경
pd.options.display.float_format = '{:.2f}'.format

# 모든 컬럼 표시
pd.set_option('display.max_columns', None)

df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.00,1,0,7.25,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.00,1,0,71.28,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.00,0,0,7.92,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.00,1,0,53.10,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.00,0,0,8.05,S,Third,man,True,NaN,Southampton,no,True


- df1에서 1, 3, 5번행을 삭제해 주세요
- df1에서 embarked, class, alone 컬럼을 삭제해 주세요
- df1의 상위 10개 행을 출력하세요

In [51]:
# df 데이터프레임 복제
df1 = df.copy()

In [52]:
df1 = df1.drop([1, 3, 5])
df1 = df1.drop(['embarked', 'class', 'alone'], axis=1)
df1.head(10)

,survived,pclass,sex,age,sibsp,parch,fare,who,adult_male,deck,embark_town,alive
0,0,3,male,22.00,1,0,7.25,man,True,NaN,Southampton,no
2,1,3,female,26.00,0,0,7.92,woman,False,NaN,Southampton,yes
4,0,3,male,35.00,0,0,8.05,man,True,NaN,Southampton,no
6,0,1,male,54.00,0,0,51.86,man,True,E,Southampton,no
7,0,3,male,2.00,3,1,21.07,child,False,NaN,Southampton,no
8,1,3,female,27.00,0,2,11.13,woman,False,NaN,Southampton,yes
9,1,2,female,14.00,1,0,30.07,child,False,NaN,Cherbourg,yes
10,1,3,female,4.00,1,1,16.70,child,False,G,Southampton,yes
11,1,1,female,58.00,0,0,26.55,woman,False,C,Southampton,yes
12,0,3,male,20.00,0,0,8.05,man,True,NaN,Southampton,no


---

iris 붓꽃 데이터셋을 활용하여 다음의 문제를 풀어주세요

- species: 붓꽃 데이터의 종류

- sepal_length: 꽃받침의 길이

- sepal_width: 꽃받침의 넓이

- petal_length: 꽃잎의 길이

- petal_width: 꽃잎의 넓이

In [53]:
iris = sns.load_dataset('iris')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.10,3.50,1.40,0.20,setosa
1,4.90,3.00,1.40,0.20,setosa
2,4.70,3.20,1.30,0.20,setosa
3,4.60,3.10,1.50,0.20,setosa
4,5.00,3.60,1.40,0.20,setosa


- sepal 컬럼을 생성하고, sepal_length 와 sepal_width를 곱한 값을 대입하세요

- 상위 5개 행만 출력해 주세요

In [54]:
iris['sepal'] = iris['sepal_length'] * iris['sepal_width']
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species,sepal
0,5.10,3.50,1.40,0.20,setosa,17.85
1,4.90,3.00,1.40,0.20,setosa,14.70
2,4.70,3.20,1.30,0.20,setosa,15.04
3,4.60,3.10,1.50,0.20,setosa,14.26
4,5.00,3.60,1.40,0.20,setosa,18.00


- petal 컬럼을 생성하고, petal_length 와 petal_width를 곱한 값을 대입하세요

- 상위 5개 행만 출력해 주세요

In [55]:
iris['petal'] = iris['petal_length'] * iris['petal_width']
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species,sepal,petal
0,5.10,3.50,1.40,0.20,setosa,17.85,0.28
1,4.90,3.00,1.40,0.20,setosa,14.70,0.28
2,4.70,3.20,1.30,0.20,setosa,15.04,0.26
3,4.60,3.10,1.50,0.20,setosa,14.26,0.30
4,5.00,3.60,1.40,0.20,setosa,18.00,0.28


- petal_length, petal_width 컬럼을 제거하세요

- 상위 5개 행만 출력해 주세요

In [56]:
iris = iris.drop(['petal_length', 'petal_width'], axis=1)
iris.head()

,sepal_length,sepal_width,species,sepal,petal
0,5.10,3.50,setosa,17.85,0.28
1,4.90,3.00,setosa,14.70,0.28
2,4.70,3.20,setosa,15.04,0.26
3,4.60,3.10,setosa,14.26,0.30
4,5.00,3.60,setosa,18.00,0.28


- species가 setosa인 꽃 중 sepal을 기준으로 내림차순 정렬 하세요.

- 상위 10개의 행만 출력하세요

In [57]:
iris.loc[iris['species'] == 'setosa'].sort_values('sepal', ascending=False).head(10)

,sepal_length,sepal_width,species,sepal,petal
15,5.70,4.40,setosa,25.08,0.60
14,5.80,4.00,setosa,23.20,0.24
33,5.50,4.20,setosa,23.10,0.28
18,5.70,3.80,setosa,21.66,0.51
32,5.20,4.10,setosa,21.32,0.15
5,5.40,3.90,setosa,21.06,0.68
16,5.40,3.90,setosa,21.06,0.52
10,5.40,3.70,setosa,19.98,0.30
48,5.30,3.70,setosa,19.61,0.30
46,5.10,3.80,setosa,19.38,0.32


sepal 컬럼의 평균과 petal 컬럼의 평균의 차이를 구하세요

In [58]:
iris['sepal'].mean() - iris['petal'].mean()

np.float64(12.028799999999997)

---

In [60]:
df2 = pd.read_csv('data/서울시자전거/seoul_bicycle.csv')
df2.head()

,대여일자,대여소번호,대여소명,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리,이용시간
0,Jan-20-2020,3,중랑센터,일일(회원),M,AGE_003,3,61.82,0.52,2230.00,75
1,Jan-20-2020,3,중랑센터,일일(회원),M,AGE_004,1,39.62,0.28,1220.00,15
2,Jan-20-2020,3,중랑센터,정기,M,AGE_005,3,430.85,4.01,17270.00,53
3,Jan-20-2020,5,상암센터 정비실,일일(회원),\N,AGE_005,2,1.79,0.02,90.00,33
4,Jan-20-2020,5,상암센터 정비실,정기,F,AGE_003,1,4501.96,45.47,196010.00,64


대여일자 컬럼에서 연도, 월, 일, 요일 데이터를 추출하여 컬럼을 추가하세요.

상위 5개 행만 출력하세요

In [61]:
df2['대여일자'] = pd.to_datetime(df2['대여일자'])
df2['연도'] = df2['대여일자'].dt.year
df2['월'] = df2['대여일자'].dt.month
df2['일'] = df2['대여일자'].dt.day
df2['요일'] = df2['대여일자'].dt.dayofweek
df2.head()

,대여일자,대여소번호,대여소명,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리,이용시간,연도,월,일,요일
0,2020-01-20,3,중랑센터,일일(회원),M,AGE_003,3,61.82,0.52,2230.00,75,2020,1,20,0
1,2020-01-20,3,중랑센터,일일(회원),M,AGE_004,1,39.62,0.28,1220.00,15,2020,1,20,0
2,2020-01-20,3,중랑센터,정기,M,AGE_005,3,430.85,4.01,17270.00,53,2020,1,20,0
3,2020-01-20,5,상암센터 정비실,일일(회원),\N,AGE_005,2,1.79,0.02,90.00,33,2020,1,20,0
4,2020-01-20,5,상암센터 정비실,정기,F,AGE_003,1,4501.96,45.47,196010.00,64,2020,1,20,0


---

In [62]:
sample = sns.load_dataset('titanic')
sample.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.00,1,0,7.25,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.00,1,0,71.28,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.00,0,0,7.92,S,Third,woman,False,NaN,Southampton,yes,True


age를 다음과 같은 구간을 가지도록 나누어 주세요

1. 0세 초과 ~ 15세 이하

2. 15세 초과 ~ 30세 이하

3. 30세 초과 ~ 45세 이하

4. 45세 초과 ~ 최대값

나눈 다음 age_bin 컬럼을 생성하여 나눈 값을 대입하고 분포를 출력하세요

cut() 함수를 활용하세요

In [68]:
sample['age_bin'] = pd.cut(sample['age'], [0, 15, 30, 45, sample['age'].max()])
sample['age_bin'].value_counts()

age_bin
(15.0, 30.0]    326
(30.0, 45.0]    202
(45.0, 80.0]    103
(0.0, 15.0]      83
Name: count, dtype: int64

나이를 3개의 균등 분할을 갖도록 구간을 나누고 분포를 출력하세요

- 나이가 어린 그룹부터 차례대로 'young', 'normal', 'old' 라벨을 부여하세요  

qcut() 함수를 활용하세요

In [69]:
sample['age_qbin'] = pd.qcut(sample['age'], 3, labels=['young', 'normal', 'old'])
sample['age_qbin'].value_counts()

age_qbin
young     246
old       236
normal    232
Name: count, dtype: int64